#### Required Installs

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install soundfile

In [ ]:
!pip install protobuf

In [ ]:
!pip install librosa

In [ ]:
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install ffmpeg

In [ ]:
!pip install torchaudio

In [ ]:
!pip install gradio

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install evaluate

In [ ]:
!pip install -U wandb

In [ ]:
!apt-get install git-lfs

In [ ]:
!git lfs install

In [1]:
# have to login to access speech-commands dataset
from huggingface_hub import login
login()

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: ptah23. Use `wandb login --relogin` to force relogin


True

# Build a music genre classifier

## Pre-trained models and datasets for audio classification

#### Minds-14

In [3]:
from datasets import load_dataset
minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

Reusing dataset minds14 (/root/.cache/huggingface/datasets/PolyAI___minds14/en-AU/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696)


In [4]:
from transformers import pipeline
classifier = pipeline("audio-classification", model="anton-l/xtreme_s_xlsr_300m_minds14")

In [ ]:
classifier(minds[0]["path"])

In [ ]:
import IPython
IPython.display.Audio(minds[0]["path"])

#### Speech commands

In [ ]:
speech_commands = load_dataset("speech_commands", "v0.02", split="validation", streaming=True)
sample = next(iter(speech_commands))
sample

In [ ]:
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
classifier = pipeline("audio-classification", model="MIT/ast-finetuned-speech-commands-v2")
classifier(sample["audio"])

#### FLEURS

In [ ]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [ ]:
sample

In [ ]:
IPython.display.Audio(data=sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
classifier = pipeline("audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id")

In [ ]:
classifier(sample["audio"])

In [ ]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
sample= next(iter(dataset))
audio_sample = sample["audio"]["array"]
sample

In [ ]:
IPython.display.Audio(data=audio_sample, rate=44_100)

In [ ]:
candidate_labels=["Sound of a dog", "Sound of vacuum cleaner"]

In [ ]:
classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")
classifier(audio_sample, candidate_labels=candidate_labels)

## Fine-tuning a model for music classification

### The Dataset

In [5]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Using custom data configuration all
Reusing dataset gtzan (/root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [6]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-52d2398c8e4ac745.arrow and /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910/cache-3bcc56e346e4d81c.arrow


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [7]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229], dtype=float32),
  'sampling_rate': 22050},
 'genre': 7}

In [8]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [9]:
# import gradio as gr
# def generate_audio():
#     example = gtzan["train"].shuffle()[0]
#     audio = example["audio"]
#     return (
#         audio["sampling_rate"],
#         audio["array"],
#     ), id2label_fn(example["genre"])


# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(4):
#             audio, label = generate_audio()
#             output = gr.Audio(audio, label=label)

# demo.launch(debug=True, share=True)

### Picking a pretrained model for audio classification

In [10]:
from transformers import AutoFeatureExtractor
model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, do_normalize=True, return_attention_mask=False)

In [11]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [12]:
from datasets import Audio
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [13]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427], dtype=float32),
  'sampling_rate': 16000},
 'genre': 7}

In [14]:
import numpy as np
sample = gtzan["train"][0]["audio"]
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [15]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt")
print(f"inputs keys: {list(inputs.keys())}")
                                                                 

inputs keys: ['input_values']


In [16]:
max_duration = 30.0
def preprocess_function(examples):
    audio_arrays=[x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), truncation=True, return_attention_mask=False)
    return inputs

In [17]:
gtzan_encoded = gtzan.map( preprocess_function, remove_columns=["audio", "file"], batched=True, num_proc=1)
gtzan_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 100
    })
})

In [18]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [19]:
id2label = { str(i): id2label_fn(i) for i in range(len(gtzan_encoded["train"].features["label"].names))}
label2id = {v: k for k,v in id2label.items()}
id2label["7"]

'pop'

In [20]:
from transformers import AutoModelForAudioClassification
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(model_id, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from transformers import TrainingArguments
model_name = model_id.split("/")[-1]
batch_size=2
gradient_accumulation_steps=4
num_train_epochs=20
%env WANDB_PROJECT=huggingface-audio-course
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    report_to="wandb",
)

env: WANDB_PROJECT=huggingface-audio-course


In [28]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
                       

In [29]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,

)
trainer.train()
#trainer.train(resume_from_checkpoint=True)

/notebooks/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan is already a clone of https://huggingface.co/ptah23/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.956900,0.646722,0.770000
2,0.544100,0.589479,0.800000
2,0.453600,0.407001,0.820000
4,0.109600,0.381224,0.890000
4,0.011600,1.166065,0.780000
6,0.016500,0.482241,0.910000
6,0.120600,0.500029,0.880000
8,0.000100,0.407390,0.890000
8,0.206800,0.476858,0.870000
10,0.000100,0.374332,0.890000


TrainOutput(global_step=2240, training_loss=0.1367254428033318, metrics={'train_runtime': 2634.3954, 'train_samples_per_second': 6.825, 'train_steps_per_second': 0.85, 'total_flos': 1.2134672243018957e+18, 'train_loss': 0.1367254428033318, 'epoch': 19.91})

In [30]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}
trainer.push_to_hub(**kwargs)

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/329M [00:00<?, ?B/s]

To https://huggingface.co/ptah23/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan
   91bb306..c1d3e49  main -> main

To https://huggingface.co/ptah23/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan
   c1d3e49..49a7388  main -> main



'https://huggingface.co/ptah23/ast-finetuned-audioset-10-10-0.4593-finetuned-gtzan/commit/c1d3e499b6653268edfeeeade1b335b5614d9c5c'